# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [17]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sys
import re
import pickle
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support,accuracy_score,label_ranking_average_precision_score
from sklearn.model_selection  import GridSearchCV

nltk.download(['punkt','stopwords','wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\binwan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\binwan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\binwan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
def load_data(db_path='workspace/data/DisasterResponse.db',tablename='disastertab'):
    """
    Function: load data from database and return X and y.
    Args:
      db_path(str): database file name included path
      tablename:(str): table name in the database file.
    Return:
      X(pd.DataFrame): messages for X
      y(pd.DataFrame): labels part in messages for y
    """
    # load data from database
    engine = create_engine('sqlite:///'+db_path)
    df=pd.read_sql_table(tablename, engine)
    X = df['message']
    # result is multiple classifty.
    y = df.iloc[:,4:]
    return X, y

In [9]:
X,y=load_data()

In [10]:
print(X[:5].values.tolist())
#need sent_tokenize to parse X.

['Weather update - a cold front from Cuba that could pass over Haiti', 'Is the Hurricane over or is it not over', 'Looking for someone but no name', 'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.', 'says: west side of Haiti, rest of the country today and tonight']


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    """
    Function: tokenize the text
    Args:  source string
    Return:
    clean_tokens(str list): clean string list
    
    """
    #normalize text
    text = re.sub(r'[^a-zA-Z0-9]',' ',text.lower())
    
    #token messages
    words = word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words("english")]
    
    #sterm and lemmatizer
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect',TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=200)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=42)
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [18]:


def display_results(Y_test, y_pred):
    
    result=precision_recall_fscore_support(Y_test, y_pred)
    for i, col in enumerate(Y_test.columns.values):
        accu=accuracy_score(Y_test.loc[:,col],y_pred[:,i])
        score = ('{}\n Accuracy:  {:.4f}   % Precision: {:.4f}   % Recall {:.4f} '.format(
                  col,accu,result[0][i],result[1][i]))
        print(score)
    avg_precision = label_ranking_average_precision_score(Y_test, y_pred)
    avg_score= ('label ranking average precision: {}'.format(avg_precision))
    print(avg_score)
    
    

In [13]:
y_pred=pipeline.predict(X_test)

In [19]:
display_results(y_test, y_pred)

C:\Users\binwan\AppData\Local\Continuum\anaconda3\envs\aind\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\binwan\AppData\Local\Continuum\anaconda3\envs\aind\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


related
 Accuracy:  0.8209   % Precision: 0.8395   % Recall 0.9460 
request
 Accuracy:  0.8972   % Precision: 0.8438   % Recall 0.4848 
offer
 Accuracy:  0.9954   % Precision: 0.0000   % Recall 0.0000 
aid_related
 Accuracy:  0.7850   % Precision: 0.7682   % Recall 0.6862 
medical_help
 Accuracy:  0.9220   % Precision: 0.7600   % Recall 0.0874 
medical_products
 Accuracy:  0.9498   % Precision: 0.7667   % Recall 0.0824 
search_and_rescue
 Accuracy:  0.9741   % Precision: 0.5000   % Recall 0.0441 
security
 Accuracy:  0.9815   % Precision: 0.3333   % Recall 0.0104 
military
 Accuracy:  0.9712   % Precision: 0.7692   % Recall 0.0633 
child_alone
 Accuracy:  1.0000   % Precision: 0.0000   % Recall 0.0000 
water
 Accuracy:  0.9578   % Precision: 0.8608   % Recall 0.4060 
food
 Accuracy:  0.9485   % Precision: 0.8601   % Recall 0.6421 
shelter
 Accuracy:  0.9397   % Precision: 0.8363   % Recall 0.4038 
clothing
 Accuracy:  0.9880   % Precision: 0.8889   % Recall 0.1143 
money
 Accuracy:  0.

In [20]:
pipeline.get_params()

{'clf': MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
            n_jobs=1),
 'clf__estimator': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
 'clf__estimator__bootstrap': True,
 'clf__estimator__class_

In [23]:
from sklearn.externals import joblib
model = joblib.load("workspace/models/classifier.pkl")
print(model.best_params_)
p1=model.best_estimator_
p1.get_params()

{'clf__estimator__min_samples_leaf': 1, 'clf__estimator__max_features': 'auto', 'vect__smooth_idf': True}


{'clf': MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
             oob_score=False, random_state=20, verbose=0, warm_start=False),
            n_jobs=1),
 'clf__estimator': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
             oob_score=False, random_state=20, verbose=0, warm_start=False),
 'clf__estimator__bootstrap': True,
 'clf__estimator__class_weight': None,
 'clf__estimato

    pipeline = Pipeline([
        ('vect',TfidfVectorizer(tokenizer=tokenize)),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=200,random_state=20)))
    ])
    parameters = {
        'clf__estimator__min_samples_leaf': [1,10],
        'clf__estimator__max_features': ['auto','log2'],
        'vect__smooth_idf':[True]
    }
    


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
    'clf__estimator__min_samples_leaf': [1,10],
    'clf__estimator__max_features': ['auto','log2'],
    'vect__smooth_idf':[True]
}

# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-1)

parameters_bak = {
    'clf__estimator__n_estimators': [200,100],
    'clf__estimator__max_depth': [3,15],
    'clf__estimator__min_samples_leaf': [1,8],
    'vect__smooth_idf': [True,False],
    'vect__sublinear_tf':[True,False]
}

In [ ]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)   

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
display_results(y_test, y_pred)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline = Pipeline([
    ('vect',TfidfVectorizer(tokenizer=tokenize)),
    ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
])

In [ ]:
pipeline.get_params()

In [ ]:
parameters = {
    'vect__smooth_idf': [True,False],
}
# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-1)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)   
display_results(y_test, y_pred)

### 9. Export your model as a pickle file

In [ ]:
with open('classifer.pkl', 'wb') as f:
    pickle.dump(cv, f)


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [21]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sys
import re
import pickle
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
from sklearn.model_selection  import GridSearchCV

nltk.download(['punkt','stopwords','wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\binwan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\binwan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\binwan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [22]:

def load_data(db_path='workspace/data/DisasterResponse.db',tablename='disastertab'):
    """
    Function: load data from database and return X and y.
    Args:
      db_path(str): database file name included path
      tablename:(str): table name in the database file.
    Return:
      X(pd.DataFrame): messages for X
      y(pd.DataFrame): labels part in messages for y
    """
    # load data from database
    engine = create_engine('sqlite:///'+db_path)
    df=pd.read_sql('SELECT * from '+tablename, engine)
    X = df['message']
    # result is multiple classifty.
    y = df.iloc[:,4:]
    return X, y

def tokenize(text):
    """
    Function: tokenize the text
    Args:  source string
    Return:
    clean_tokens(str list): clean string list
    
    """
    #normalize text
    text = re.sub(r'[^a-zA-Z0-9]',' ',text.lower())
    
    #token messages
    words = word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words("english")]
    
    #sterm and lemmatizer
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

def build_model(): 
    """
    Function: build model that consist of pipeline
    Args:
      N/A
    Return
      cv(model): Grid Search model 
    """
    
    pipeline = Pipeline([
        ('vect',TfidfVectorizer(tokenizer=tokenize)),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=200,random_state=20)))
    ]) 
    
    parameters = {
        'clf__estimator__criterion':['entropy']
    }
    
    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-1)
    return cv

def evaluate_model(y_test, y_pred):
    result=precision_recall_fscore_support(Y_test, y_pred)
    for i, col in enumerate(category_names):
        accu=accuracy_score(Y_test.loc[:,col],y_pred[:,i])
        print('{}\n Accuracy: {:.4f}    % Precision: {:.4f}   % Recall {:.4f} '.format(
            col,accu,result[0][i],result[1][i]))
    
def save_model(cv):
    """
    Function: save model as pickle file.
    Args:
      cv:target model
    Return:
      N/A
    """
    with open('classifer.pkl', 'wb') as f:
        pickle.dump(cv, f)    
def main(): 
    print("Load data")
    X,y=load_data()
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=42)
    print("Build model")
    model=build_model()
    print("Train model")
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    print("Evaluation model")
    evaluate_model(y_test, y_pred)
    print("Save model")
    save_model(model)


In [ ]:
if __name__ == "__main__":
    main()